# Netflix Analysis

Crisp-DM Cycle 1

---

## 0. Preparation

### 0.1 Imports

In [4]:
import pandas as pd

### 0.2 Data

In [5]:
# Paths
homepath = '/Users/guttofranca/Repos/netflix-analysis/netflix_analysis'
datapath = homepath + '/data/raw/'

# Data

# CA
df_CA = pd.read_csv(datapath + 'CAvideos.csv', encoding='utf-8')
df_CA['country'] = 'Canada'

# DE   
df_DE = pd.read_csv(datapath + 'DEvideos.csv', encoding='utf-8')
df_DE['country'] = 'Germany'

# FR
df_FR = pd.read_csv(datapath + 'FRvideos.csv', encoding='utf-8')
df_FR['country'] = 'France'

# GB
df_GB = pd.read_csv(datapath + 'GBvideos.csv', encoding='utf-8')
df_GB['country'] = 'Great Britain'

# IN
df_IN = pd.read_csv(datapath + 'INvideos.csv', encoding='utf-8')
df_IN['country'] = 'India'

# JP
df_JP = pd.read_csv(datapath + 'JPvideos.csv', encoding='ISO-8859-1')
df_JP['country'] = 'Japan'

# KR
df_KR = pd.read_csv(datapath + 'KRvideos.csv', encoding='ISO-8859-1')
df_KR['country'] = 'South Korea'

# MX
df_MX = pd.read_csv(datapath + 'MXvideos.csv', encoding='ISO-8859-1')
df_MX['country'] = 'Mexico'

# RU
df_RU = pd.read_csv(datapath + 'RUvideos.csv', encoding='ISO-8859-1')
df_RU['country'] = 'Russia'

# US
df_US = pd.read_csv(datapath + 'USvideos.csv', encoding='utf-8')
df_US['country'] = 'United States'

# Using only utf-8 encoded files
df_raw = pd.concat([df_CA, df_DE, df_FR, df_GB, df_IN, df_US], ignore_index=True).reset_index(drop=True)

# Saving the raw data as pickle
df_raw.to_pickle(homepath + '/data/processed/df_raw.pkl')

### 0.3 Functions

In [6]:
def fix_tags(tags):
    """
    Fixes the formatting of tags by removing extra spaces and adding double quotes.

    Args:
        tags (str): A string containing tags separated by '|'.

    Returns:
        str: A string with tags adjusted by removing extra spaces and adding double quotes.

    Example:
        >>> fix_tags("action |  comedy | drama")
        '"action" | "comedy" | "drama"'
    """

    separated_tags = tags.split('|')
    adjusted_tags = ['"' + tag.replace('"', '').strip() + '"' for tag in separated_tags]
    
    return ' | '.join(adjusted_tags)



## 1. Data Description

In [7]:
df1 = pd.read_pickle(homepath + '/data/processed/df_raw.pkl')

### 1.1 Data Dimensions

In [8]:
print(f'Number of rows: {df1.shape[0]} \nNumber of columns: {df1.shape[1]}')

Number of rows: 239662 
Number of columns: 17


### 1.2 Changing Dtypes

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239662 entries, 0 to 239661
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                239662 non-null  object
 1   trending_date           239662 non-null  object
 2   title                   239662 non-null  object
 3   channel_title           239662 non-null  object
 4   category_id             239662 non-null  int64 
 5   publish_time            239662 non-null  object
 6   tags                    239662 non-null  object
 7   views                   239662 non-null  int64 
 8   likes                   239662 non-null  int64 
 9   dislikes                239662 non-null  int64 
 10  comment_count           239662 non-null  int64 
 11  thumbnail_link          239662 non-null  object
 12  comments_disabled       239662 non-null  bool  
 13  ratings_disabled        239662 non-null  bool  
 14  video_error_or_removed  239662 non-n

`Data Types`

`video_id` : String

`trending_date`: Datetime

`title`: String

`channel_title`: String

`category_id` : Int

`publish_time`: Timestamp

`tags`: String

`views`: Int

`likes`: Int

`dislikes`: Int

`comment_count`: Int

`thumbnail_link`: String

`comments_disabled`: Boolean

`ratings_disabled`: Boolean

`video_error_or_removed`: Boolean

`description`: String

`country`: String

In [10]:
# Transformations

# Changing the columns to string
columns_to_string = [['video_id', 'title', 'channel_title', 'tags', 'thumbnail_link', 'description', 'country']]

for column in columns_to_string:
    df1[column] = df1[column].astype('string')

# Changing the columns to datetime
df1['trending_date'] = pd.to_datetime(df1['trending_date'], format='%y.%d.%m')

# Changing the columns to timestamp
df1['publish_time'] = pd.to_datetime(df1['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [11]:
df1.dtypes

video_id                  string[python]
trending_date             datetime64[ns]
title                     string[python]
channel_title             string[python]
category_id                        int64
publish_time              datetime64[ns]
tags                      string[python]
views                              int64
likes                              int64
dislikes                           int64
comment_count                      int64
thumbnail_link            string[python]
comments_disabled                   bool
ratings_disabled                    bool
video_error_or_removed              bool
description               string[python]
country                   string[python]
dtype: object

### 1.3 Fixing Inconsistences

In [12]:
df1.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,2017-11-14,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 17:00:03,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,Canada
1,0dBIkQ4Mz1M,2017-11-14,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,Canada
2,5qpjK5DgCt4,2017-11-14,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Canada
3,d380meD0W0M,2017-11-14,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Canada
4,2Vv-BfVoq4g,2017-11-14,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,Canada


In [13]:
# Replace "|" with " | " and split the last tag
df1['tags'] = df1['tags'].apply(fix_tags)

### 1.4 Unique Values

In [14]:
df1.nunique()

video_id                   94007
trending_date                205
title                      94704
channel_title              17798
category_id                   18
publish_time               86983
tags                       74046
views                     178399
likes                      66911
dislikes                   15413
comment_count              23219
thumbnail_link             94179
comments_disabled              2
ratings_disabled               2
video_error_or_removed         2
description                82201
country                        6
dtype: int64

### 1.5 NULL

In [15]:
df1.isnull().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               7503
country                      0
dtype: int64

In [16]:
df_null_description = df1[df1['description'].isnull()]
df_null_description.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
98,lHcXhBojpeQ,2017-11-14,三屆TVB視帝，拋棄10年青梅竹馬髮妻，為娶小三還不惜與母絕交！,明星百曉生,22,2017-11-12 12:49:50,"""[none]""",88061,47,58,17,https://i.ytimg.com/vi/lHcXhBojpeQ/default.jpg,False,False,False,<NA>,Canada
115,_QHEPUKBn9g,2017-11-14,‘Too Alabama for Alabama’: SNL brutally mocks ...,MAXI POWER,23,2017-11-12 05:01:18,"""[none]""",20163,44,34,8,https://i.ytimg.com/vi/_QHEPUKBn9g/default.jpg,False,False,False,<NA>,Canada
128,JdHovvb0KD4,2017-11-14,Pres. Trump: Putin's Denials Of Interference I...,The View,25,2017-11-13 16:10:21,"""donald trump"" | ""vladimir putin"" | ""the view""...",72253,1217,146,569,https://i.ytimg.com/vi/JdHovvb0KD4/default.jpg,False,False,False,<NA>,Canada
165,OA0vhAKUD0g,2017-11-14,Outlander S03E09 i'm melting with you'',Ella bell,1,2017-11-12 22:33:48,"""[none]""",32941,149,2,5,https://i.ytimg.com/vi/OA0vhAKUD0g/default.jpg,False,False,False,<NA>,Canada
179,psDauoxUj9A,2017-11-14,شاهد انبهار الاعلام المصري بتأهل المغرب لكأس ...,Info Press,17,2017-11-11 22:04:37,"""[none]""",624328,7100,398,971,https://i.ytimg.com/vi/psDauoxUj9A/default.jpg,False,False,False,<NA>,Canada


There are 7503 NA rows in the `description` column, to quick fix this, I'll replace it with a description called: No description provided.

In [17]:
df1['description'] = df1['description'].fillna('No description provided.')

df1.isnull().sum()

video_id                  0
trending_date             0
title                     0
channel_title             0
category_id               0
publish_time              0
tags                      0
views                     0
likes                     0
dislikes                  0
comment_count             0
thumbnail_link            0
comments_disabled         0
ratings_disabled          0
video_error_or_removed    0
description               0
country                   0
dtype: int64

### 1.6 Duplicated Values

In [18]:
df1.duplicated().sum()

4482

In [19]:
df1[df1.duplicated()].head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
156188,RUCXD3_wW2w,2018-05-14,New Hulu Show - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",723495,6362,1589,420,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156189,p8npDG2ulKQ,2018-05-14,BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...,ibighit,10,2018-05-06 15:00:02,"""BIGHIT"" | ""빅히트"" | ""방탄소년단"" | ""BTS"" | ""BANGTAN""...",26912663,2636004,27675,366899,https://i.ytimg.com/vi/p8npDG2ulKQ/default.jpg,False,False,False,BTS (방탄소년단) LOVE YOURSELF 轉 Tear 'Singularity'...,Great Britain
156190,aixso4N2vhI,2018-05-14,President Trump Gives Remarks on the Joint Com...,The White House,25,2018-05-08 18:45:37,"""[none]""",89715,2806,488,1657,https://i.ytimg.com/vi/aixso4N2vhI/default.jpg,False,False,False,The White House,Great Britain
156191,bu0m_UdtoaU,2018-05-14,Serious Questions: Avengers Infinity War,Screen Junkies,1,2018-05-06 17:00:03,"""screenjunkies"" | ""screen junkies"" | ""serious ...",443621,12892,3829,2275,https://i.ytimg.com/vi/bu0m_UdtoaU/default.jpg,False,False,False,After watching Avengers: Infinity War we have ...,Great Britain
156192,i-G1hy73Mb8,2018-05-14,Eurovision Song Contest 2018 - Opening Ceremon...,Eurovision Song Contest,24,2018-05-06 19:46:34,"""Eurovision Song Contest"" | ""2018"" | ""Lisbon"" ...",698827,11721,461,1175,https://i.ytimg.com/vi/i-G1hy73Mb8/default.jpg,False,False,False,"At 18:00 CEST today, the Blue Carpet ceremony ...",Great Britain


An example:

In [20]:
df1[df1['video_id'] == 'RUCXD3_wW2w']


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
156074,RUCXD3_wW2w,2018-05-14,New Hulu Show - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",723495,6362,1589,420,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156188,RUCXD3_wW2w,2018-05-14,New Hulu Show - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",723495,6362,1589,420,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156246,RUCXD3_wW2w,2018-05-15,Handmaids in the City - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",1094378,8781,2234,600,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156320,RUCXD3_wW2w,2018-05-15,Handmaids in the City - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",1094378,8781,2234,600,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156404,RUCXD3_wW2w,2018-05-16,Handmaids in the City - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",1250529,9870,2437,649,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain
156552,RUCXD3_wW2w,2018-05-16,Handmaids in the City - SNL,Saturday Night Live,24,2018-05-13 04:59:28,"""SNL"" | ""Saturday Night Live"" | ""SNL Season 43...",1250529,9870,2437,649,https://i.ytimg.com/vi/RUCXD3_wW2w/default.jpg,False,False,False,From the executive producer of Sex and the Cit...,Great Britain


The columns are all the same in the example, so I will drop them all.

In [21]:
df1 = df1.drop_duplicates()